In [4]:
import pandas as pd
import numpy as np
from uuid import uuid4
import os
import json
import sys

In [5]:
sample_match_path = '../data/t20s_json/211028.json'
sample_match = pd.read_json(sample_match_path)

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [6]:
# Option 1: First load the JSON using json module to inspect the structure
import json

with open(sample_match_path, 'r') as f:
    data = json.load(f)
    
meta = data['meta']
info = data['info']
innings = data['innings']

In [7]:
print(f"meta: {meta}")
print(f"info: {info}")

meta: {'data_version': '1.0.0', 'created': '2013-02-22', 'revision': 1}
info: {'balls_per_over': 6, 'city': 'Southampton', 'dates': ['2005-06-13'], 'gender': 'male', 'match_type': 'T20', 'match_type_number': 2, 'officials': {'match_referees': ['JJ Crowe'], 'tv_umpires': ['MR Benson'], 'umpires': ['NJ Llong', 'JW Lloyds']}, 'outcome': {'by': {'runs': 100}, 'winner': 'England'}, 'overs': 20, 'player_of_match': ['KP Pietersen'], 'players': {'Australia': ['AC Gilchrist', 'ML Hayden', 'A Symonds', 'MJ Clarke', 'MEK Hussey', 'RT Ponting', 'DR Martyn', 'B Lee', 'JN Gillespie', 'MS Kasprowicz', 'GD McGrath'], 'England': ['ME Trescothick', 'GO Jones', 'A Flintoff', 'KP Pietersen', 'MP Vaughan', 'PD Collingwood', 'AJ Strauss', 'VS Solanki', 'J Lewis', 'D Gough', 'SJ Harmison']}, 'registry': {'people': {'A Flintoff': 'ddc0828d', 'A Symonds': 'bd77eb62', 'AC Gilchrist': '2b6e6dec', 'AJ Strauss': 'b68d14a9', 'B Lee': 'dd09ff8e', 'D Gough': 'fcbf5a30', 'DR Martyn': '69762509', 'GD McGrath': 'ee7d0c8

In [5]:
len(innings)

2

In [6]:
innings[0].keys()

dict_keys(['team', 'overs', 'powerplays'])

In [7]:
innings[1].keys()

dict_keys(['team', 'overs', 'powerplays', 'target'])

In [8]:
df = pd.DataFrame(innings[0]['overs'])
df


,over,deliveries
0,0,"[{'batter': 'ME Trescothick', 'bowler': 'B Lee..."
1,1,"[{'batter': 'ME Trescothick', 'bowler': 'GD Mc..."
2,2,"[{'batter': 'ME Trescothick', 'bowler': 'B Lee..."
3,3,"[{'batter': 'GO Jones', 'bowler': 'GD McGrath'..."
4,4,"[{'batter': 'ME Trescothick', 'bowler': 'B Lee..."
5,5,"[{'batter': 'A Flintoff', 'bowler': 'MS Kaspro..."
6,6,"[{'batter': 'KP Pietersen', 'bowler': 'JN Gill..."
7,7,"[{'batter': 'ME Trescothick', 'bowler': 'MS Ka..."
8,8,"[{'batter': 'KP Pietersen', 'bowler': 'JN Gill..."
9,9,"[{'batter': 'ME Trescothick', 'bowler': 'MS Ka..."


In [9]:
innings[0]['overs'][0]['deliveries'][0]

{'batter': 'ME Trescothick',
 'bowler': 'B Lee',
 'non_striker': 'GO Jones',
 'runs': {'batter': 0, 'extras': 0, 'total': 0}}

In [18]:
df_row = {
    # Match identifiers
    'game_id': None,
    'date': None,
    'venue': None,
    'location': None,
    'gender': None,
    'match_type': None,
    'innings': None,  # 1st or 2nd innings
    
    # Team info
    'batting_team': None,
    'bowling_team': None,
    'batting_team_players': None,
    'bowling_team_players': None,
    
    # Over/Ball info
    'over': None,
    'ball_in_over': None,  # 1-6
    
    # Players
    'batter': None,
    'bowler': None,
    'non_striker': None,
    
    # Runs breakdown
    'runs_batter': None,    # Runs scored by batter
    'extras': None,         # Extra runs
    'total': None,          # Total runs for delivery
    
    # Extra details
    'is_wicket': False,     # Boolean
    'wicket_type': None,    # caught/bowled/lbw etc
    'fielder': None,        # If caught/run-out
    
    # Target (for 2nd innings)
    'target_runs': None,
    'target_overs': None,
    
    # Match situation
    'current_runs': None,  # Team score before this ball
    'current_wickets': None,       # Wickets fallen before this ball
    'powerplay': False,     # Boolean - is it powerplay?
    'extras_details': None,
}


In [36]:
def process_match(data):
    match_df = pd.DataFrame(columns=df_row.keys())

    meta = data['meta']
    info = data['info']
    all_innings = data['innings']
#-----------------------------------------
    game_id = uuid4()
    date = info['dates'][0]
    venue =  info['venue']
    location = info['city'] if 'city' in info else None # Some matches don't have a city but keeping it may be userful later
    gender = info['gender']
    match_type = info['match_type']
    innings = None
    teams = info['teams']
#-----------------------------------------
    innings_count = 0
    for inning in all_innings:
        innings_count += 1
        runs_this_innings = 0
        wickets_this_innings = 0

        batting_team = inning['team']
        bowling_team = list(set(teams) - {batting_team})[0]

        batting_team_players = info['players'][batting_team]
        batting_team_players_ids = [info['registry']['people'][player] for player in batting_team_players]
        bowling_team_players = info['players'][bowling_team]
        bowling_team_players_ids = [info['registry']['people'][player] for player in bowling_team_players]
        
        # While I am only looking at t20 matches, I'd like to keep it more flexible for other match formats
        powerplay_info = inning['powerplays']
        powerplay_start = str(powerplay_info[0]['from']).split('.')
        powerplay_end = str(powerplay_info[0]['to']).split('.')

        if 'target' in inning:
            target_runs = inning['target']['runs']
            target_overs = inning['target']['overs']
        else:
            target_runs = None
            target_overs = None

        for over in inning['overs']:
            over_num = over['over']
            delivery_count = 0
            for delivery in over['deliveries']:
                delivery_count += 1

                powerplay = True if over_num >= int(powerplay_start[0]) and over_num <= int(powerplay_end[0]) and delivery_count >= int(powerplay_start[1]) and delivery_count <= int(powerplay_end[1]) else False

                batter = info['registry']['people'][delivery['batter']]
                bowler = info['registry']['people'][delivery['bowler']]
                non_striker = info['registry']['people'][delivery['non_striker']]

                runs_batter = delivery['runs']['batter']
                extras = delivery['runs']['extras']
                total = delivery['runs']['total']
                runs_this_innings += total

                if 'wickets' in delivery:
                    wickets_this_innings += 1
                    is_wicket = True
                    wicket_type = delivery['wickets'][0]['kind']
                    player_out = info['registry']['people'][delivery['wickets'][0]['player_out']]
                    if 'fielders' in delivery['wickets'][0]:
                        fielder = info['registry']['people'][delivery['wickets'][0]['fielders'][0]]
                    else:
                        fielder = None
                else:
                    is_wicket = False
                    wicket_type = None
                    player_out = None
                    fielder = None

                over_row = {
                    # Match identifiers
                    'game_id': game_id,
                    'date': date,
                    'venue': venue,
                    'location': location,
                    'gender': gender,
                    'match_type': match_type,
                    'innings': innings_count,  # 1st or 2nd innings
                    
                    # Team info
                    'batting_team': batting_team,
                    'bowling_team': bowling_team,
                    'batting_team_players': batting_team_players_ids,
                    'bowling_team_players': bowling_team_players_ids,
                    
                    # Over/Ball info
                    'over': over_num,
                    'ball_in_over': delivery_count,  # 1-6
                    
                    # Players
                    'batter': batter,
                    'bowler': bowler,
                    'non_striker': non_striker,
                    
                    # Runs breakdown
                    'runs_batter': runs_batter,    # Runs scored by batter
                    'extras': extras,         # Extra runs
                    'total': total,          # Total runs for delivery
                    
                    # Extra details
                    'is_wicket': is_wicket,     # Boolean
                    'wicket_type': wicket_type,    # caught/bowled/lbw etc
                    'fielder': fielder,        # If caught/run-out
                    'player_out': player_out,
                    
                    # Target (for 2nd innings)
                    'target_runs': target_runs,
                    'target_overs': target_overs,
                    
                    # Match situation
                    'current_runs': runs_this_innings,  # Team score before this ball
                    'current_wickets': wickets_this_innings,       # Wickets fallen before this ball
                    'powerplay': powerplay,     # Boolean - is it powerplay?
                    'extras_details': delivery.get('extras'),
                }
                match_df = pd.concat([match_df, pd.DataFrame([over_row])], ignore_index=True)
    return match_df
    


In [19]:
def process_match(data):
    # Create a list to store rows instead of using concat
    match_rows = []
    
    # Extract match info once
    info = data['info']
    game_id = uuid4()
    match_info = {
        'game_id': game_id,
        'date': info['dates'][0],
        'venue': info['venue'],
        'location': info.get('city'),  # Using .get() is safer
        'gender': info['gender'],
        'match_type': info['match_type']
    }
    
    teams = info['teams']
    registry = info['registry']['people']  # Cache the registry lookup
    
    for innings_count, inning in enumerate(data['innings'], 1):
        # Pre-calculate innings info
        batting_team = inning['team']
        bowling_team = list(set(teams) - {batting_team})[0]
        
        # Pre-process player IDs
        batting_team_players_ids = [registry[player] for player in info['players'][batting_team]]
        bowling_team_players_ids = [registry[player] for player in info['players'][bowling_team]]
        
        # Process powerplay info once
        powerplay_info = inning['powerplays'][0]
        pp_start = [int(x) for x in str(powerplay_info['from']).split('.')]
        pp_end = [int(x) for x in str(powerplay_info['to']).split('.')]
        
        # Target info
        target_info = inning.get('target', {})
        target_runs = target_info.get('runs')
        target_overs = target_info.get('overs')
        
        runs_this_innings = 0
        wickets_this_innings = 0
        
        # Process deliveries
        for over in inning['overs']:
            over_num = over['over']
            for delivery_count, delivery in enumerate(over['deliveries'], 1):
                # Calculate powerplay
                powerplay = (over_num >= pp_start[0] and over_num <= pp_end[0] and 
                           delivery_count >= pp_start[1] and delivery_count <= pp_end[1])
                
                # Process runs
                runs = delivery['runs']
                total = runs['total']
                runs_this_innings += total
                
                # Process wickets
                wicket_info = delivery.get('wickets', [{}])[0] if 'wickets' in delivery else {}
                is_wicket = bool(wicket_info)
                if is_wicket:
                    wickets_this_innings += 1
                
                # Create row dict
                row = {
                    **match_info,  # Unpack match info
                    'innings': innings_count,
                    'batting_team': batting_team,
                    'bowling_team': bowling_team,
                    'batting_team_players': batting_team_players_ids,
                    'bowling_team_players': bowling_team_players_ids,
                    'over': over_num,
                    'ball_in_over': delivery_count,
                    'batter': registry[delivery['batter']],
                    'bowler': registry[delivery['bowler']],
                    'non_striker': registry[delivery['non_striker']],
                    'runs_batter': runs['batter'],
                    'extras': runs['extras'],
                    'total': total,
                    'is_wicket': is_wicket,
                    'wicket_type': wicket_info.get('kind'),
                    'fielder': registry[wicket_info['fielders'][0]['name']] if 'fielders' in wicket_info else None,
                    'player_out': registry[wicket_info['player_out']] if 'player_out' in wicket_info else None,
                    'target_runs': target_runs,
                    'target_overs': target_overs,
                    'current_runs': runs_this_innings,
                    'current_wickets': wickets_this_innings,
                    'powerplay': powerplay,
                    'extras_details': delivery.get('extras'),
                }
                match_rows.append(row)
    
    # Create DataFrame once at the end
    return pd.DataFrame(match_rows)

In [13]:
df = process_match(data)
df.head(10)

,game_id,date,venue,location,gender,match_type,innings,batting_team,bowling_team,batting_team_players,...,total,is_wicket,wicket_type,fielder,player_out,target_runs,target_overs,current_runs,current_wickets,powerplay
0,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,0,False,None,None,None,NaN,NaN,0,0,True
1,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,1,False,None,None,None,NaN,NaN,1,0,True
2,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,0,False,None,None,None,NaN,NaN,1,0,True
3,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,0,False,None,None,None,NaN,NaN,1,0,True
4,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,0,False,None,None,None,NaN,NaN,1,0,True
5,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,1,False,None,None,None,NaN,NaN,2,0,True
6,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,2,False,None,None,None,NaN,NaN,4,0,False
7,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,0,False,None,None,None,NaN,NaN,4,0,True
8,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,0,False,None,None,None,NaN,NaN,4,0,True
9,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,1,False,None,None,None,NaN,NaN,5,0,True


In [16]:
df[df['is_wicket'] == True].head()

,game_id,date,venue,location,gender,match_type,innings,batting_team,bowling_team,batting_team_players,...,total,is_wicket,wicket_type,fielder,player_out,target_runs,target_overs,current_runs,current_wickets,powerplay
24,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,0,True,caught,8d0ea930,2e929b99,NaN,NaN,28,1,True
38,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,0,True,caught,bd77eb62,ddc0828d,NaN,NaN,49,2,True
68,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,0,True,caught,d8699ab7,39f01cdb,NaN,NaN,100,3,False
71,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,0,True,caught,7d415ea5,6e1b1ec0,NaN,NaN,102,4,False
82,c7e4c2a0-ea58-40ca-b6f8-7d918a0885b1,2005-06-13,The Rose Bowl,Southampton,male,T20,1,England,Australia,"[ea42ddb9, 2e929b99, ddc0828d, 39f01cdb, 6e1b1...",...,0,True,caught,48fd7349,ea42ddb9,NaN,NaN,109,5,False


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   game_id               215 non-null    object 
 1   date                  215 non-null    object 
 2   venue                 215 non-null    object 
 3   location              215 non-null    object 
 4   gender                215 non-null    object 
 5   match_type            215 non-null    object 
 6   innings               215 non-null    int64  
 7   batting_team          215 non-null    object 
 8   bowling_team          215 non-null    object 
 9   batting_team_players  215 non-null    object 
 10  bowling_team_players  215 non-null    object 
 11  over                  215 non-null    int64  
 12  ball_in_over          215 non-null    int64  
 13  batter                215 non-null    object 
 14  bowler                215 non-null    object 
 15  non_striker           2

In [20]:
def process_all_matches(path_to_data_directory):
    files = os.listdir(path_to_data_directory)
    all_matches_df = pd.DataFrame()

    for file in files:
        try:
            with open(os.path.join(path_to_data_directory, file), 'r') as f:
                data = json.load(f)
                match_df = process_match(data)
                all_matches_df = pd.concat([all_matches_df, match_df], ignore_index=True)
        except Exception as e:
            print(f"Error processing match: {file}")
            print(f"Error: {e}")
            print("\n\n")
            continue
    return all_matches_df

all_matches_df = process_all_matches('../data/t20s_json')
all_matches_df.info()

Error processing match: 1442989.json
Error: 'powerplays'



Error processing match: 1424765.json
Error: 'name'





/var/folders/5b/kfhm4mss6sjdr7rf8618rz1w0000gq/T/ipykernel_73785/607605127.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_matches_df = pd.concat([all_matches_df, match_df], ignore_index=True)


Error processing match: 902653.json
Error: 'powerplays'



Error processing match: 1422804.json
Error: 'powerplays'



Error processing match: 1400975.json
Error: 'powerplays'



Error processing match: 1405327.json
Error: 'powerplays'



Error processing match: 1187669.json
Error: 'powerplays'



Error processing match: 1459720.json
Error: 'powerplays'



Error processing match: 551596.json
Error: 'name'



Error processing match: 1188380.json
Error: 'powerplays'



Error processing match: 1190607.json
Error: 'powerplays'



Error processing match: 1370791.json
Error: 'powerplays'



Error processing match: 1423474.json
Error: 'powerplays'



Error processing match: 1415711.json
Error: 'powerplays'



Error processing match: 533292.json
Error: 'powerplays'



Error processing match: 533284.json
Error: 'powerplays'



Error processing match: 1187680.json
Error: 'powerplays'



Error processing match: 1183544.json
Error: 'powerplays'



Error processing match: 1187679.json
Error: 'power

In [22]:
all_matches_df['game_id'].nunique()

3793

3793 games out of a total of 3835 is pretty good going.

There's quite a few matches that have no location, but I think this is okay, I doubt we really need it, if we want to group by location venue would probably be a better choice anyway.

Of 858370 balls, there are 47481 wickets, so about 1 in 20. Of those, 29896 were due to a catch.
Target runs is included in about half of the balls, which looks about right.

In [21]:
all_matches_df.to_csv('../data/saved_data/all_matches.csv', index=False)

---

In [31]:
matches_df = pd.read_csv('../data/saved_data/all_matches.csv')

In [32]:
matches_df['batter'].nunique()

5399

---